Aplicando restrições na busca
=============================



## Introdução



Muitos problemas de otimização com relevância científica têm uma ou mais `restrições` que devem ser levadas em consideração na hora de resolver o problema.

Lembra do `problema da mochila` que vimos em Lógica Computacional? Era um problema de otimização onde queríamos maximizar o valor dos itens colocados na mochila enquanto observávamos a restrição do peso total dos itens (do contrário, a mochila rasgava).

Uma forma de considerar essas restrições nos problemas é aplicando uma `penalidade` na função objetivo.

Vamos pensar como seria essa penalidade no problema da mochila: a função objetivo é maximizar o valor dos itens na mochila, então é um problema de maximização. A função objetivo pode ser a soma dos itens da mochila. Se fosse só isso, teríamos

$$
f = \sum_{i, i \in \mathrm{mochila}}\mathrm{valor}(i)
$$

No entanto, apenas essa função não resolve o problema! Precisamos levar em consideração o limite de peso da mochila! Para isso, penalizamos a função objetivo levando em consideração essa restrição:

$f=\begin{cases}
0.01 & \textrm{se peso > limite da mochila}\\
\sum_{i,i\in\mathrm{mochila}}(\mathrm{valor}(i)) & \textrm{se peso} \leq \textrm{limite da mochila}
\end{cases}$

Agora finalmente podemos seguir em frente e resolver o problema.



## Reflexões



Se usarmos a equação de $f$ acima, qual será o valor de $f$ caso não exista uma solução para um certo problema da mochila?

Na equação de $f$ acima nós usamos um valor práximo de zero para indicar que uma restrição do problema não foi satisfeita. Você consegue pensar em outra estratégia para penalizar soluções inválidas?



## Objetivo



Encontrar uma solução para o problema da mochila usando algoritmos genéticos. Considere que existem 10 itens diferentes (com pesos e valores diferentes) disponíveis para serem escolhidos.



## Descrição do problema



No problema da mochila você tem um número $n$ de itens disponíveis, cada um com um peso e um valor associado. Sua mochila tem a capacidade de carregar um número $p$ de quilogramas, sendo que mais que isso faz com que sua mochila rasgue e todos os itens dentro dela caiam no chão e se quebrem de maneira catastrófica (indesejado). Sua tarefa é encontrar um conjunto de itens (considerando os $n$ disponíveis) que maximize o valor contido dentro da mochila, porém que tenham um peso dentro da capacidade da mesma.



## Importações



In [2]:
import random as rd

from constantes import DIC_OBJ as OBJETOS

from funcoes import backpack_values
from funcoes import fitness_pop_mochila
from funcoes import new_pop_mochila
from funcoes import roull_sel_max_mochila
from funcoes import crossover
from funcoes import mutation_mochila

## Códigos e discussão



In [26]:
### CONSTANTES

# relacionadas à busca
TAM_POP = 20
NUM_GEN = 1000
PC = 0.5
PM = 0.05

# relacionadas ao problema a ser resolvido
LIM = 5
NUM_OBJETOS = len(OBJETOS)
ORDEM_DOS_NOMES = list(sorted(OBJETOS.keys()))

In [27]:
# Funções locais

def fitness_pop(pop):
    return fitness_pop_mochila(
        pop, OBJETOS, LIM, ORDEM_DOS_NOMES
    )

In [36]:
pop = new_pop_mochila(TAM_POP, NUM_OBJETOS)

melhor_fitness_ja_visto = -float("inf")
melhor_individuo_ja_visto = [0] * NUM_OBJETOS

for n in range(NUM_GEN):

    # Roleta
    fitness = fitness_pop(pop)
    pop = roull_sel_max_mochila(pop, fitness)

    # Crossover
    p1_list = pop[0::2]
    p2_list = pop[1::2]

    c = 0
    for p1, p2 in zip(p1_list, p2_list):
        filho1, filho2 = crossover(p1, p2, PC)
        pop[c] = filho1
        pop[c + 1] = filho2
        c += 2

    # Mutação

    for i in range(len(pop)):
        curr_ind = pop[i]
        pop[i] = mutation_mochila(curr_ind, PM)

    # Hall da fama
    fitness = fitness_pop(pop)
    maior_fitness = max(fitness)
    posicao = fitness.index(maior_fitness)
    ind = pop[posicao].copy()
    valor, peso = backpack_values(ind, OBJETOS, ORDEM_DOS_NOMES)
    if maior_fitness > melhor_fitness_ja_visto and peso <= LIM:
        melhor_fitness_ja_visto = maior_fitness
        melhor_individuo_ja_visto = ind
        print(f"Maior valor: {valor} | Peso: {peso}")


# Melhor individuo
print()
print("Você deve pegar os seguintes itens:")
for pega_ou_nao, item in zip(melhor_individuo_ja_visto, ORDEM_DOS_NOMES):
    if pega_ou_nao == 1:
        print("+", item)
print()
valor_total, peso_total = backpack_values(
    melhor_individuo_ja_visto, OBJETOS, ORDEM_DOS_NOMES
)
print(
    f"Valor total: {valor_total}\n"
    f"Peso total: {peso_total}"
)

Maior valor: 1180 | Peso: 4.47
Maior valor: 1210 | Peso: 4.9
Maior valor: 1280 | Peso: 5.0
Maior valor: 1310 | Peso: 4.8500000000000005
Maior valor: 1350 | Peso: 4.87
Maior valor: 1470 | Peso: 4.67
Maior valor: 1520 | Peso: 4.970000000000001

Você deve pegar os seguintes itens:
+ Chinelo ninja com ponta de aço
+ Espelho que mostra seu reflexo como um animal
+ Fone de ouvido em forma de orelhas de gato
+ Gravata que toca música quando apertada
+ Máscara com estampa de personagem do pica-pau
+ Máscara de dormir que parece olhos abertos
+ Pendrive em forma de sushi
+ Pijama para cães com asas
+ Roupa de banho comestível
+ Touca de natação à prova de choque elétrico
+ Tênis com rodinhas para adultos

Valor total: 1520
Peso total: 4.970000000000001


**Desafio**: resolva o experimento considerando uma busca em grade para encontrar a melhor resposta.



## Conclusão



### Efetividade do código:
De maneira semelhante ao que ocorre no algotimo do caixeiro viajante, uma busca em grade teria aumento exponencial no custo computacional, nesse prisma o algoritmo genético se mostra extremamente útil, retornando uma lista de itens - não necessariamente a melhor - que se aproxima da resolução do problema.  
Com o aumento do número de gerações, a eficácia do algoritmo genético pode ser melhor; Outra solução possível é tornar o algoritmo um problema de minimazação que tende a 0, tendo como critério de parada não mais o número de gerações, e sim quando a menor *fitness function* se aproxima muito (ou se iguala) à zero.

## Playground

